In [1]:
import gc
from all_stand_var import conv_dict, lab_cols, vent_cols3
from all_own_funct import memory_downscale,memory_upscale
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import seaborn as sns
from seaborn import heatmap
from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import kurtosis
from matplotlib.backends.backend_pdf import PdfPages
import keras.backend as K
from RNN_LTSM_CHD import return_loaded_model
import pickle
import locale
import LR_build_CHD as pp
import tables
import pickle

from sklearn.metrics import roc_curve, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report, confusion_matrix


locale.setlocale(locale.LC_ALL, 'fr_FR')
output_folder = os.path.join(os.getcwd(), 'Results_LR_RF_CHD_v2','Static')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    

f = open(os.path.join(output_folder,'ran_for.sav'), 'rb')
best_rf=pickle.load(f)
f.close()

f = open(os.path.join(output_folder,'log_reg.sav'), 'rb')
best_lr=pickle.load(f)
f.close()

C:\Users\berend\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\berend\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [ ]:
import datetime as dt
import tables
from sklearn.preprocessing import StandardScaler



df=pd.read_hdf(os.path.join('./Results_CHD/', 'processed_df.h5'),key='df',mode='r')
#df=memory_downscale(df)
print(df.info())
from sklearn.preprocessing import StandardScaler
df=df.reset_index(drop=False)
df.info()
def index_1hr(group):
    if len(group) > 720:
        group=group.sort_values('pat_datetime')
        group=group.tail(720)
    group['tim']=pd.date_range(start='1/1/2018', periods=len(group), freq='1min')
    grouped = pd.Grouper(key='tim', freq='1H')
    group['idx_1hr']=group.groupby(grouped,sort=False).ngroup().add(1)
    #group['mis']=(group['pat_datetime'].max()-group['pat_datetime'].min()).total_seconds()/60
    del group['tim']
    return group
#grouped = pd.Grouper(key='pat_datetime', freq='60min',label='left',convention='start')
#df['idx_1hr']=df.groupby(['Adnum','pat_hosp_id','AdmissionDate','DischargeDate'],sort=False,as_index=False).ngroup().add(1)
df=df.groupby('Adnum',sort=False,as_index=False).apply(index_1hr)
df['idx_1hr']=df['idx_1hr'].astype(str)
print(df['idx_1hr'].unique())
#df[['mis']]=StandardScaler().fit_transform(df[['mis']])
df=df.set_index(['pat_hosp_id','OK_datum'])



pat=pd.read_excel(r'Results_CHD\admissiondate_CHD0.xlsx',
                    parse_dates=['OK_datum'],index_col=0)
group=pat.groupby('pat_hosp_id',sort=False).max().reset_index()
group.drop_duplicates('pat_hosp_id',inplace=True)

df_train,df_val,df_test=pp.split_stratified_into_train_val_test(group, stratify_colname='Reintub',
                                         frac_train=0.7, frac_val=0.1, frac_test=0.2,
                                         random_state=1)

train_pat=df_train['pat_hosp_id'].unique()
test_pat=df_test['pat_hosp_id'].unique()
val_pat=df_val['pat_hosp_id'].unique()

df_train = df[df.index.get_level_values('pat_hosp_id').isin(train_pat)]
df_val = df[df.index.get_level_values('pat_hosp_id').isin(val_pat)]
df_test = df[df.index.get_level_values('pat_hosp_id').isin(test_pat)]
print(df_train.info())
print(df_test.info())

def x_modelling(df):
    temp = df[['Age', 'Adnum', 'idx_1hr','pat_weight_act','Diagnose']]
    df = df.drop(['Age','pat_weight_act','Extubation_date','level_0','pat_datetime', 'Reintub','Diagnose'], axis=1)
    df = df.groupby(['Adnum', 'idx_1hr'], sort=False).agg(['mean', 'std'])
    df.columns = ["_".join(a) for a in df.columns.to_flat_index()]
    df = df.stack().unstack([2, 1])
    df.columns = ["_".join(a) for a in df.columns.to_flat_index()]
    df = df.reset_index().drop_duplicates().set_index('Adnum')
    df = df.fillna(method='ffill').fillna(method='bfill')

    temp = temp.groupby('Adnum', sort=False).agg(['mean'])
    temp.columns = ["_".join(a) for a in temp.columns.to_flat_index()]
    temp = temp.reset_index().drop_duplicates().set_index('Adnum')
    temp = temp[~temp.index.duplicated(keep='last')]
    df = df.merge(temp, right_index=True, left_index=True, how='left')
    return df
X_TRAIN=memory_upscale(x_modelling(df_train))
X_TEST=memory_upscale(x_modelling(df_test))
X_VAL=memory_upscale(x_modelling(df_val))
print(X_TRAIN.info())
print(X_TEST.info())
X_TRAIN=X_TRAIN.fillna(method='ffill').fillna(method='bfill')
X_VAL=X_VAL.fillna(method='ffill').fillna(method='bfill')
X_TEST=X_TEST.fillna(method='ffill').fillna(method='bfill')

X_TRAIN=X_TRAIN.append(X_VAL)

In [ ]:
def y_modelling(df):
    y = df[['Reintub', 'Adnum', 'idx_1hr']]
    y = y.groupby(['Adnum', 'idx_1hr'], sort=False).agg(['max'])
    y.columns = ["_".join(a) for a in y.columns.to_flat_index()]
    y.reset_index(drop=True, inplace=True, level='idx_1hr')
    y = y.reset_index().drop_duplicates().set_index(
        ['Adnum'])
    y = y[~y.index.duplicated(keep='last')]
    return y
Y_TRAIN=y_modelling(df_train)
Y_TEST=y_modelling(df_test)
Y_VAL=y_modelling(df_val)



In [4]:
clf=best_lr
f = open(os.path.join(output_folder, 'cols.pkl'), 'rb')
float_columns = pickle.load(f)
f.close()

print(float_columns)
try:
    pdf = PdfPages(os.path.join(output_folder,f"Figures_importance_prob.pdf"))
except PermissionError:
    os.remove(os.path.join(output_folder,f"Figures_importance_prob.pdf"))

try:
    coefs = clf.coef_.flatten()
except:
    coefs=clf.feature_importances_

# Zip coefficients and names together and make a DataFrame
zipped = zip(float_columns, coefs)
df = pd.DataFrame(zipped, columns=["feature", "value"])
# Sort the features by the absolute value of their coefficient
df["abs_value"] = df["value"].apply(lambda x: abs(x))
df["colors"] = df["value"].apply(lambda x: "green" if x > 0 else "red")
df = df.sort_values("abs_value", ascending=False)


import seaborn as sns
fig, ax = plt.subplots(1, 1,figsize=(16,16))
sns.barplot(x="feature",
            y="value",
            data=df.head(20),
            palette=df.head(20)["colors"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=65, fontsize=10)
ax.set_title("Top 20 Features, LR", fontsize=12)
ax.set_ylabel("Coef", fontsize=12)
ax.set_xlabel("Feature Name", fontsize=12)
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig) 


['mis_mean', 'pat_weight_act_mean', 'Age_mean', 'Diagnose_mean']


In [5]:
clf=best_rf

try:
    coefs = clf.coef_.flatten()
except:
    coefs=clf.feature_importances_

# Zip coefficients and names together and make a DataFrame
zipped = zip(float_columns, coefs)
df = pd.DataFrame(zipped, columns=["feature", "value"])
# Sort the features by the absolute value of their coefficient
df["abs_value"] = df["value"].apply(lambda x: abs(x))
df["colors"] = df["value"].apply(lambda x: "green" if x > 0 else "red")
df = df.sort_values("abs_value", ascending=False)


import seaborn as sns
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
sns.barplot(x="feature",
            y="value",
            data=df.head(30),
            palette=df.head(30)["colors"])
ax.set_xticklabels(ax.get_xticklabels(), rotation=65, fontsize=10)
ax.set_title("Top 20 Features, RF", fontsize=12)
ax.set_ylabel("Coef", fontsize=12)
ax.set_xlabel("Feature Name", fontsize=12)
print(df["feature"].head(20).tolist())
plt.tight_layout()
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig) 
pdf.close()

['pat_weight_act_mean', 'mis_mean', 'Diagnose_mean', 'Age_mean']


In [ ]:
print(X_TEST.info())

In [ ]:
prediction=best_lr.predict_proba(X_TEST)[:,1]
#print(prediction)
true_0 = (Y_TEST == 0.0)
true_1 = (Y_TEST == 1.0)
df_test=pd.DataFrame()
df_test['prob']=best_lr.predict_proba(X_TEST)[:,1]
df_test['true']=Y_TEST

plt.figure(figsize=(15,7))
plt.hist(df_test['prob'].loc[df_test['true']==0], bins=50, label='Negatives')
plt.hist(df_test['prob'].loc[df_test['true']==1], bins=50, label='Positives', alpha=0.7, color='r')
plt.xlabel('Probability of being Positive Class', fontsize=25)
plt.ylabel('Number of records in each bucket', fontsize=25)
plt.title('Probability distribution of best LR')
plt.legend(fontsize=15)
plt.tick_params(axis='both', labelsize=25, pad=5)
plt.show()
plt.tight_layout()
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig) 

In [ ]:
prediction=best_rf.predict_proba(X_TRAIN)[:,1]
true_0 = (Y_TEST == 0.0)
true_1 = (Y_TEST == 1.0)
df_test=pd.DataFrame()
df_test['prob']=best_rf.predict_proba(X_TEST)[:,1]
df_test['true']=Y_TEST

plt.figure(figsize=(15,7))
plt.hist(df_test['prob'].loc[df_test['true']==0], bins=50, label='Negatives')
plt.hist(df_test['prob'].loc[df_test['true']==1], bins=50, label='Positives', alpha=0.8, color='r')
plt.xlabel('Probability of being Positive Class', fontsize=25)
plt.ylabel('Number of records in each bucket', fontsize=25)
plt.title('Probability distribution of best RF')
plt.legend(fontsize=15)
plt.tick_params(axis='both', labelsize=25, pad=5)
plt.show()
fig=plt.gcf()
pdf.savefig(fig)
plt.close(fig)
pdf.close()